# Zero shot classification of Jow ingredients into Agribalyse subcategories/categories
This notebook is motivated by the fact that some Jow ingredients cannot be matched with Agribalyse ingredients (e.g. "Ricotta" because there is no "Ricotta"-like cheese in the Agribalyse database, see `[Genevieve] ingredients_scores__process.ipynb`). The idea is to classify those ingredients into categories of Agribalyse ingredients (e.g. "Fromages" for "Ricotta") and then to calculate the mean environmental score for each category.

## Import packages

In [5]:
import pandas as pd
import numpy as np
import re
import time
import seaborn as sns
import itertools

## Google colab settings
Show the gpu infos and change if needed in "Execution" panel.   
Using GPUs is not compulsory here. I moved to Google Colab because my computer is slow.

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jul 10 14:07:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Import from Google Drive.  
Here the folder 'carbondiet4GD' only contains the folder data/ with subfolders Recipes/ and Tools/ (others are not used).

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/carbondiet4GD.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')     # create a copy directly in colab for efficiency reasons

Mounted at /content/drive


**Indicate the path where the folder data/ is located.**

In [8]:
#data_path = '../'
data_path = 'carbondiet4GD/'

## Load Agribalyse data and extract the list of ingredient subcategories

In [9]:
data_AGB = pd.read_excel(data_path+"data/Agribalyse/AGRIBALYSE3.1_produits alimentaires_2.xlsm", \
                                     sheet_name = 'Synthese', skiprows = 1, header = [0, 1])

In [10]:
data_AGB.head(2)

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,Effets toxicologiques sur la santé humaine : substances cancérogènes,Acidification terrestre et eaux douces,Eutrophisation eaux douces,Eutrophisation marine,Eutrophisation terrestre,Écotoxicité pour écosystèmes aquatiques d'eau douce,Utilisation du sol,Épuisement des ressources eau,Épuisement des ressources énergétiques,Épuisement des ressources minéraux
,Code\nAGB,Code\nCIQUAL,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,code saison (0 : hors saison ; 1 : de saison ; 2 : mix de consommation FR),code avion (1 : par avion),Livraison,Matériau d'emballage,...,CTUh/kg de produit,mol H+ eq/kg de produit,kg P eq/kg de produit,kg N eq/kg de produit,mol N eq/kg de produit,CTUe/kg de produit,Pt/kg de produit,m3 depriv./kg de produit,MJ/kg de produit,kg Sb eq/kg de produit
0,11084,11084,aides culinaires et ingrédients divers,algues,"Agar (algue), cru","Seaweed, agar, raw",2,0,Ambiant (long),LDPE,...,6.482667e-09,0.099733,0.001868,0.01328,0.127350,44.555153,26.476190,3.102487,315.057170,0.000088
1,11023,11023,aides culinaires et ingrédients divers,herbes,"Ail séché, poudre","Garlic, powder, dried",2,0,Ambiant (long),Verre,...,4.662163e-10,0.006222,0.000124,0.00232,0.012695,5.832120,20.223652,2.520528,12.665447,0.000007


We remove ligatures (not sure it's compulsory)

In [11]:
REGEX_REPLACEMENTS = [(r"\u0153", "oe"), \
                      (r"\u0152", "Oe")]

def remove_ligatures(transcript):
    for old, new in REGEX_REPLACEMENTS:
        transcript = re.sub(old, new, transcript, flags=re.IGNORECASE)
    return transcript

for col in data_AGB.columns:
    if pd.api.types.is_object_dtype(data_AGB[col].dtypes):   # if the column type is object
        data_AGB[col] = data_AGB[col].apply(lambda s: remove_ligatures(str(s)))

Extract the list of categories and the list of subcategories of Agribalyse ingredients

In [12]:
# Flatten the MultiIndex into a Single Index
data_AGB.columns = ['_'.join(col) if type(col) is tuple else col for col in data_AGB.columns.values]

# Rename relevant columns
data_AGB = data_AGB.rename(columns = {"Unnamed: 2_level_0_Groupe d'aliment": "Groupe d'aliment",
                                      "Unnamed: 3_level_0_Sous-groupe d'aliment": "Sous-groupe d'aliment"
                                      })

# List of Agribalyse categories
AGB_category_list = data_AGB["Groupe d'aliment"].unique().tolist()

# List of Agribalyse subcategories
AGB_subcategory_list = data_AGB["Sous-groupe d'aliment"].unique().tolist()


In [13]:
AGB_category_list

['aides culinaires et ingrédients divers',
 'aliments infantiles',
 'boissons',
 'entrées et plats composés',
 'fruits, légumes, légumineuses et oléagineux',
 'glaces et sorbets',
 'lait et produits laitiers',
 'matières grasses',
 'produits céréaliers',
 'produits sucrés',
 'viandes, oeufs, poissons']

In [14]:
AGB_subcategory_list

['algues',
 'herbes',
 'ingrédients divers',
 'épices',
 'aides culinaires',
 'sauces',
 'sels',
 'condiments',
 'denrées destinées à une alimentation particulière',
 'céréales et biscuits infantiles',
 'laits et boissons infantiles',
 'desserts infantiles',
 'petits pots salés et plats infantiles',
 'boisson alcoolisées',
 'boissons sans alcool',
 'eaux',
 'plats composés',
 'feuilletées et autres entrées',
 'soupes',
 'plats végétariens',
 'sandwichs',
 'pizzas, tartes et crêpes salées',
 'salades composées et crudités',
 'fruits',
 'fruits à coque et graines oléagineuses',
 'légumes',
 'pommes de terre et autres tubercules',
 'légumineuses',
 'glaces',
 'desserts glacés',
 'sorbets',
 'fromages',
 'produits laitiers frais et assimilés',
 'crèmes et spécialités à base de crème',
 'laits',
 'beurres',
 'huiles et graisses végétales',
 'autres matières grasses',
 'huiles de poissons',
 'margarines',
 'pâtes, riz et céréales',
 'farines et pâtes à tarte',
 'gâteaux et pâtisseries',
 'pa

## Load Jow ingredients

In [15]:
jow_data = pd.read_excel(data_path+"data/Recipes/Jow_Agribalyse_ingredients_scores.xlsx")
jow_data.head()

,Unnamed: 0,"JOW ingredients (simple, fr)","AGB ingredients (simple, fr)",Code AGB,Mean PEF,Mean CO2,Mean Ozone_1,Mean Rad,Mean Ozone_2,Mean Particules,...,Mean Toxic_2,Mean Acid,Mean Eutro_1,Mean Eutro_2,Mean Eutro_3,Mean Ecotoxic,Mean Sol,Mean Ressource_1,Mean Ressource_2,Mean Ressource_3
0,0,Boeuf,Boeuf,"['6231', '6230', '6212', '6211', '6202', '6208...",3.242932,33.410098,7.906164e-07,1.517672,0.049556,3.093358e-06,...,1.612534e-08,0.459327,0.001986,0.103966,2.023236,160.192085,2338.165351,7.217654,94.138578,0.000061
1,1,Emmental,"['Emmental ou emmenthal', 'Emmental ou emmenth...","['12115', '12118']",0.593099,6.209078,2.410305e-07,0.668786,0.010852,4.610472e-07,...,3.805963e-09,0.063426,0.000849,0.021059,0.269473,63.473248,334.889540,2.151466,33.300605,0.000016
2,2,Sauce Teriyaki,Sauce soja,['11104'],0.099203,0.743117,1.465743e-07,0.694233,0.002102,3.848904e-08,...,6.416595e-10,0.004271,0.000163,0.002924,0.014445,9.289838,31.861036,0.389270,21.895355,0.000006
3,3,Riz,Riz,['9121'],0.975725,2.757780,1.402258e-07,0.163422,0.009600,1.807163e-07,...,1.421574e-09,0.029098,0.000553,0.021770,0.110556,60.410509,65.083849,78.981602,22.371707,0.000015
4,5,Gnocchi,"['Gnocchi à la semoule', 'Gnocchi à la pomme d...","['26265', '25479', '26264', '25510']",0.107950,0.854534,8.972039e-08,0.463608,0.002575,5.413420e-08,...,7.256132e-10,0.006592,0.000148,0.003738,0.024992,8.204390,45.734868,0.821471,18.930097,0.000005


In [16]:
jow_ingredient_list = jow_data["JOW ingredients (simple, fr)"].values.tolist()
jow_ingredient_list[:10]

['Boeuf',
 'Emmental',
 'Sauce Teriyaki',
 'Riz',
 'Gnocchi',
 'Epinard',
 'Gorgonzola',
 'Crème liquide',
 'Lentilles corail',
 'Tomate']

## Zero-shot classification of Jow ingredients into Agribalyse subcategories

See docs here : https://thedatafrog.com/fr/articles/pre-trained-transformers-hugging-face/

In [13]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00


In [1]:
! pip install tokenizer datasets sentencepiece protobuf==3.20.0

In [2]:
from transformers import pipeline

In [3]:
classifier = pipeline("zero-shot-classification", model="BaptisteDoyen/camembert-base-xnli")

In [17]:
jow_ingredient = "Boeuf"
classifier(jow_ingredient, AGB_subcategory_list)

{'sequence': 'Boeuf',
 'labels': ['oeufs',
  'charcuteries',
  'fromages',
  'fruits',
  'laits',
  'viandes cuites',
  'poissons cuits',
  'herbes',
  'légumes',
  'légumineuses',
  'condiments',
  'viandes crues',
  'substituts de charcuterie',
  'sandwichs',
  'plats composés',
  'aides culinaires',
  'beurres',
  'denrées destinées à une alimentation particulière',
  'produits laitiers frais et assimilés',
  'boisson alcoolisées',
  'substituts de viande',
  'eaux',
  'autres matières grasses',
  'mollusques et crustacés cuits',
  'produits à base de poissons et produits de la mer',
  'sauces',
  'crèmes et spécialités à base de crème',
  'poissons crus',
  'desserts infantiles',
  'chocolats et produits à base de chocolat',
  'plats végétariens',
  'mollusques et crustacés crus',
  'sels',
  'boissons sans alcool',
  'soupes',
  'autres produits à base de viande',
  'confiseries non chocolatées',
  'huiles de poissons',
  'feuilletées et autres entrées',
  'algues',
  'ingrédients

In [18]:
jow_ingredient = "Gingembre"
classifier(jow_ingredient, AGB_subcategory_list)

{'sequence': 'Gingembre',
 'labels': ['herbes',
  'algues',
  'épices',
  'fruits',
  'légumes',
  'condiments',
  'fromages',
  'légumineuses',
  'substituts de charcuterie',
  'substituts de viande',
  'boisson alcoolisées',
  'plats composés',
  'ingrédients divers',
  'sorbets',
  'desserts glacés',
  'eaux',
  'denrées destinées à une alimentation particulière',
  'viandes cuites',
  'charcuteries',
  'aides culinaires',
  'sauces',
  'feuilletées et autres entrées',
  'autres matières grasses',
  'sels',
  'mollusques et crustacés cuits',
  'chocolats et produits à base de chocolat',
  'soupes',
  'glaces',
  'margarines',
  'crèmes et spécialités à base de crème',
  'autres produits à base de viande',
  'mollusques et crustacés crus',
  'viandes crues',
  'desserts infantiles',
  'oeufs',
  'beurres',
  'produits à base de poissons et produits de la mer',
  'poissons crus',
  'sandwichs',
  'poissons cuits',
  'confiseries non chocolatées',
  'confitures et assimilés',
  'huiles

It does not work very well ... Let's try to classify into categories and then into subcategories

In [23]:
jow_ingredient = "Boeuf"
clf = classifier(jow_ingredient, AGB_category_list)
clf

{'sequence': 'Boeuf',
 'labels': ['boissons',
  'produits céréaliers',
  'produits sucrés',
  'aliments infantiles',
  'matières grasses',
  'viandes, oeufs, poissons',
  'lait et produits laitiers',
  'aides culinaires et ingrédients divers',
  'entrées et plats composés',
  'fruits, légumes, légumineuses et oléagineux',
  'glaces et sorbets'],
 'scores': [0.18410778045654297,
  0.16431696712970734,
  0.14193572103977203,
  0.13656248152256012,
  0.12741777300834656,
  0.07147146761417389,
  0.061749737709760666,
  0.041539546102285385,
  0.035753898322582245,
  0.021262450143694878,
  0.013882208615541458]}

In [24]:
jow_ingredient = "Emmental"
clf = classifier(jow_ingredient, AGB_category_list)
clf

{'sequence': 'Emmental',
 'labels': ['boissons',
  'produits sucrés',
  'entrées et plats composés',
  'produits céréaliers',
  'aliments infantiles',
  'matières grasses',
  'aides culinaires et ingrédients divers',
  'lait et produits laitiers',
  'glaces et sorbets',
  'viandes, oeufs, poissons',
  'fruits, légumes, légumineuses et oléagineux'],
 'scores': [0.41605761647224426,
  0.11585748940706253,
  0.11156192421913147,
  0.08310917019844055,
  0.08167234808206558,
  0.07989293336868286,
  0.03959883004426956,
  0.03677365928888321,
  0.01626805029809475,
  0.009822613559663296,
  0.009385356679558754]}

This is worse ... it does not find the matching (main) category in Agribalyse database

### Test with another zero-shot classification model

In [29]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [26]:
jow_ingredient = "Boeuf"
clf_res = classifier(jow_ingredient, AGB_subcategory_list)
clf_res

{'sequence': 'Boeuf',
 'labels': ['plats végétariens',
  'fromages',
  'chocolats et produits à base de chocolat',
  'poissons cuits',
  'plats composés',
  'aides culinaires',
  'produits à base de poissons et produits de la mer',
  'boissons sans alcool',
  'ingrédients divers',
  'boisson alcoolisées',
  'poissons crus',
  'viandes cuites',
  'confiseries non chocolatées',
  'substituts de viande',
  'denrées destinées à une alimentation particulière',
  'oeufs',
  'charcuteries',
  'sels',
  'produits laitiers frais et assimilés',
  'viandes crues',
  'substituts de charcuterie',
  'huiles et graisses végétales',
  'crèmes et spécialités à base de crème',
  'desserts glacés',
  'petits pots salés et plats infantiles',
  'pains et viennoiseries',
  'légumineuses',
  'algues',
  'soupes',
  'mollusques et crustacés cuits',
  'autres matières grasses',
  'desserts infantiles',
  'condiments',
  'huiles de poissons',
  'autres produits à base de viande',
  'mollusques et crustacés crus

In [30]:
jow_ingredient = "Boeuf"
clf_res = classifier(jow_ingredient, AGB_category_list)
clf_res

{'sequence': 'Boeuf',
 'labels': ['matières grasses',
  'entrées et plats composés',
  'aides culinaires et ingrédients divers',
  'boissons',
  'aliments infantiles',
  'produits sucrés',
  'produits céréaliers',
  'lait et produits laitiers',
  'fruits, légumes, légumineuses et oléagineux',
  'viandes, oeufs, poissons',
  'glaces et sorbets'],
 'scores': [0.3451918959617615,
  0.19976668059825897,
  0.12888765335083008,
  0.08589677512645721,
  0.07832065969705582,
  0.039100538939237595,
  0.03370477259159088,
  0.02725979872047901,
  0.025396570563316345,
  0.01875615306198597,
  0.017718566581606865]}

In [28]:
jow_ingredient = "Emmental"
clf_res = classifier(jow_ingredient, AGB_subcategory_list)
clf_res

{'sequence': 'Emmental',
 'labels': ['fromages',
  'chocolats et produits à base de chocolat',
  'épices',
  'plats végétariens',
  'plats composés',
  'huiles et graisses végétales',
  'herbes',
  'confiseries non chocolatées',
  'légumineuses',
  'ingrédients divers',
  'aides culinaires',
  'légumes',
  'fruits',
  'boisson alcoolisées',
  'algues',
  'produits à base de poissons et produits de la mer',
  'boissons sans alcool',
  'fruits à coque et graines oléagineuses',
  'condiments',
  'denrées destinées à une alimentation particulière',
  'soupes',
  'substituts de viande',
  'pains et viennoiseries',
  'gâteaux et pâtisseries',
  'desserts glacés',
  'céréales et biscuits infantiles',
  'confitures et assimilés',
  'huiles de poissons',
  'beurres',
  'produits laitiers frais et assimilés',
  'pâtes, riz et céréales',
  'sauces',
  'sels',
  'desserts infantiles',
  'sucres, miels et assimilés',
  'substituts de charcuterie',
  'eaux',
  'sorbets',
  'crèmes et spécialités à b

In [31]:
jow_ingredient = "Emmental"
clf_res = classifier(jow_ingredient, AGB_category_list)
clf_res

{'sequence': 'Emmental',
 'labels': ['produits céréaliers',
  'lait et produits laitiers',
  'matières grasses',
  'produits sucrés',
  'boissons',
  'entrées et plats composés',
  'aides culinaires et ingrédients divers',
  'fruits, légumes, légumineuses et oléagineux',
  'aliments infantiles',
  'glaces et sorbets',
  'viandes, oeufs, poissons'],
 'scores': [0.2996889352798462,
  0.23614242672920227,
  0.17147432267665863,
  0.12392633408308029,
  0.1052325889468193,
  0.02725626528263092,
  0.013583311811089516,
  0.008846057578921318,
  0.00656531099230051,
  0.005725048948079348,
  0.0015593520365655422]}

### Test with another zero-shot classification model

In [32]:
classifier = pipeline("zero-shot-classification", model="morit/french_xlm_xnli")

In [33]:
jow_ingredient = "Boeuf"
clf_res = classifier(jow_ingredient, AGB_subcategory_list)
clf_res

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Boeuf',
 'labels': ['oeufs',
  'ingrédients divers',
  'laits',
  'beurres',
  'boisson alcoolisées',
  'plats composés',
  'crèmes et spécialités à base de crème',
  'épices',
  'fromages',
  'herbes',
  'algues',
  'condiments',
  'boissons sans alcool',
  'légumes',
  'substituts de charcuterie',
  'poissons cuits',
  'légumineuses',
  'sorbets',
  'confiseries non chocolatées',
  'glaces',
  'confitures et assimilés',
  'denrées destinées à une alimentation particulière',
  'chocolats et produits à base de chocolat',
  'substituts de viande',
  'charcuteries',
  'aides culinaires',
  'sauces',
  'eaux',
  'feuilletées et autres entrées',
  'sandwichs',
  'poissons crus',
  'laits et boissons infantiles',
  'margarines',
  'autres matières grasses',
  'viandes cuites',
  'sels',
  'desserts glacés',
  'viandes crues',
  'soupes',
  'salades composées et crudités',
  'produits laitiers frais et assimilés',
  'fruits',
  'huiles de poissons',
  'produits à base de poisso

In [34]:
jow_ingredient = "Boeuf"
clf_res = classifier(jow_ingredient, AGB_category_list)
clf_res

{'sequence': 'Boeuf',
 'labels': ['boissons',
  'produits sucrés',
  'matières grasses',
  'aliments infantiles',
  'entrées et plats composés',
  'produits céréaliers',
  'aides culinaires et ingrédients divers',
  'lait et produits laitiers',
  'viandes, oeufs, poissons',
  'fruits, légumes, légumineuses et oléagineux',
  'glaces et sorbets'],
 'scores': [0.24409830570220947,
  0.11543002724647522,
  0.09423431009054184,
  0.09304389357566833,
  0.09106190502643585,
  0.08706599473953247,
  0.0819326862692833,
  0.05603053420782089,
  0.04956050217151642,
  0.04629484936594963,
  0.04124705120921135]}

In [35]:
jow_ingredient = "Emmental"
clf_res = classifier(jow_ingredient, AGB_subcategory_list)
clf_res

{'sequence': 'Emmental',
 'labels': ['ingrédients divers',
  'plats composés',
  'épices',
  'boisson alcoolisées',
  'boissons sans alcool',
  'crèmes et spécialités à base de crème',
  'algues',
  'fromages',
  'légumes',
  'condiments',
  'laits',
  'substituts de viande',
  'denrées destinées à une alimentation particulière',
  'substituts de charcuterie',
  'herbes',
  'glaces',
  'sels',
  'légumineuses',
  'sauces',
  'beurres',
  'confitures et assimilés',
  'fruits',
  'chocolats et produits à base de chocolat',
  'sorbets',
  'confiseries non chocolatées',
  'eaux',
  'charcuteries',
  'feuilletées et autres entrées',
  'viandes cuites',
  'aides culinaires',
  'margarines',
  'poissons crus',
  'poissons cuits',
  'viandes crues',
  'salades composées et crudités',
  'sandwichs',
  'huiles et graisses végétales',
  'desserts glacés',
  'soupes',
  'laits et boissons infantiles',
  'mollusques et crustacés crus',
  'mollusques et crustacés cuits',
  'autres matières grasses',

In [36]:
jow_ingredient = "Emmental"
clf_res = classifier(jow_ingredient, AGB_category_list)
clf_res

{'sequence': 'Emmental',
 'labels': ['entrées et plats composés',
  'aides culinaires et ingrédients divers',
  'produits sucrés',
  'matières grasses',
  'boissons',
  'produits céréaliers',
  'aliments infantiles',
  'fruits, légumes, légumineuses et oléagineux',
  'lait et produits laitiers',
  'glaces et sorbets',
  'viandes, oeufs, poissons'],
 'scores': [0.1460353136062622,
  0.13608911633491516,
  0.13525770604610443,
  0.12038570642471313,
  0.11072807759046555,
  0.08602054417133331,
  0.06396675109863281,
  0.059987083077430725,
  0.05993455648422241,
  0.05373421683907509,
  0.027860956266522408]}

### Test with another zero-shot classification model

In [47]:
classifier = pipeline(
    task='zero-shot-classification',
    model="cmarkea/distilcamembert-base-nli",
    tokenizer="cmarkea/distilcamembert-base-nli"
)

In [48]:
jow_ingredient = "Boeuf"
clf_res = classifier(jow_ingredient, AGB_subcategory_list)
clf_res

{'sequence': 'Boeuf',
 'labels': ['oeufs',
  'ingrédients divers',
  'eaux',
  'plats composés',
  'charcuteries',
  'fruits',
  'laits',
  'fromages',
  'condiments',
  'beurres',
  'légumes',
  'épices',
  'chocolats et produits à base de chocolat',
  'substituts de charcuterie',
  'crèmes et spécialités à base de crème',
  'substituts de viande',
  'algues',
  'herbes',
  'confitures et assimilés',
  'boisson alcoolisées',
  'autres produits à base de viande',
  'denrées destinées à une alimentation particulière',
  'aides culinaires',
  'sorbets',
  'margarines',
  'sandwichs',
  'sauces',
  'viandes cuites',
  'produits laitiers frais et assimilés',
  'légumineuses',
  'autres matières grasses',
  'viandes crues',
  'huiles de poissons',
  'glaces',
  'soupes',
  'sels',
  'poissons cuits',
  'poissons crus',
  'feuilletées et autres entrées',
  'laits et boissons infantiles',
  'petits pots salés et plats infantiles',
  'boissons sans alcool',
  'desserts infantiles',
  'pains et

In [49]:
jow_ingredient = "Boeuf"
clf_res = classifier(jow_ingredient, AGB_category_list)
clf_res

{'sequence': 'Boeuf',
 'labels': ['boissons',
  'lait et produits laitiers',
  'entrées et plats composés',
  'produits céréaliers',
  'matières grasses',
  'aides culinaires et ingrédients divers',
  'produits sucrés',
  'aliments infantiles',
  'viandes, oeufs, poissons',
  'fruits, légumes, légumineuses et oléagineux',
  'glaces et sorbets'],
 'scores': [0.207646906375885,
  0.14603501558303833,
  0.1434052735567093,
  0.10194910317659378,
  0.09676183760166168,
  0.08907805383205414,
  0.08242306113243103,
  0.051368944346904755,
  0.041104916483163834,
  0.027130216360092163,
  0.013096708804368973]}

In [ ]:
jow_ingredient = "Emmental"
clf_res = classifier(jow_ingredient, AGB_subcategory_list)
clf_res

In [51]:
jow_ingredient = "Emmental"
clf_res = classifier(jow_ingredient, AGB_category_list)
clf_res

{'sequence': 'Emmental',
 'labels': ['boissons',
  'produits céréaliers',
  'entrées et plats composés',
  'produits sucrés',
  'aides culinaires et ingrédients divers',
  'aliments infantiles',
  'matières grasses',
  'lait et produits laitiers',
  'fruits, légumes, légumineuses et oléagineux',
  'glaces et sorbets',
  'viandes, oeufs, poissons'],
 'scores': [0.2321479171514511,
  0.1676301211118698,
  0.15429279208183289,
  0.10070236027240753,
  0.08140566200017929,
  0.0802774503827095,
  0.07319288700819016,
  0.04904060438275337,
  0.025938738137483597,
  0.0227479450404644,
  0.0126234395429492]}

## Conclusion
It does not work ... I've done only a few direct tests, maybe there are some tricks to improve the predictions, but the present results are clearly bad.